# Fetch and Classification Testing Script

This Jupyter Notebook contains Testing Functionality for the Coding IxD:Consumptution Project

## Imports

In [1]:
import requests
import random
import json
import uuid
import pandas as pd
from time import sleep
import openfoodfacts as off
from itertools import takewhile
from IPython.display import display,JSON,HTML
from pandas.io.json import json_normalize
from enum import Enum
from urllib.request import urlopen

## Functions

In [42]:
def _baseUrl(api_version):
    return f'https://world.openfoodfacts.org/api/v{api_version}'

def getProduct(ean : str,api_version=2):
    
    url = f'{_baseUrl(api_version)}/product/{ean}.json'
    
    r = requests.get(url)
    
    return r.json() if r.status_code == 200 else {}

def generateRandomShoppingList(min_len=1,max_len=100):
    # TODO: replace magic number at generator cutoff
    eans = [product['code'] for product in limited_generator(off.products.get_all_by_country('de'),max_len*3) if len(product['code'])==13]
    return random.sample(eans,random.randint(min_len,max_len))

def limited_generator(gen, limit):
    for i, item in enumerate(gen):
        if i >= limit:
            break
        yield item

def post_ean_list(eans):
    url = "http://127.0.0.1:8000/fetch/LIST/"
    headers = {'content-type': 'application/json'}
    payload = json.dumps({"eans": eans})
    response = requests.post(url, data=payload, headers=headers)
    
    if response.status_code == 200:
        return response.text
    else:
        return "Request failed with status code: " + str(response.status_code)

### Filters

In [117]:
class NutriScore(Enum):
    A = 'A'
    B = 'B'
    C = 'C'
    D = 'D'
    E = 'E'
    
def contains_ingredients_analysis_tag(json,tag : str):
    # TODO: change systematic to incoprorate 'non-x' substring parsing
    if 'ingredients_analysis_tags' in json['product'].keys():
        for _tag in json['product']['ingredients_analysis_tags']:
            if tag in _tag:
                return True
            return False
    
def filter_palm_oil_free(json) -> bool:
    # TODO: add differentiating tags
    return contains_ingredients_analysis_tag(json,'palm-oil-free')
            
def filter_fair_trade(json) -> bool:
    # TODO: add multivaried parsing
    return 'Fairtrade International' in json['product']['labels'] if 'labels' in json['product'].keys() else None

def filter_bio(json) -> bool:
    # TODO: add multivaried parsing
    return 'Bio' in json['product']['labels'] if 'labels' in json['product'].keys() else None

def filter_vegan(json) -> bool:
    return contains_ingredients_analysis_tag(json,'vegan')

def filter_vegetarian(json) -> bool:
    return contains_ingredients_analysis_tag(json,'vegetarian')

def filter_low_food_processing(json) -> bool:
    pass

def filter_nutri_score(json) -> NutriScore:
    return json['product']['nutriscore_grade'] if 'nutriscore_grade' in json['product'].keys() else None

def filter_packaging(json):
    return json['product']['packaging'] if 'packaging' in json['product'].keys() else None


def filter_transport(json):
    pass

def filter_full_filtering(json):
    return {
        'palm-oil-free' : filter_palm_oil_free(json),
        'fair-trade' : filter_fair_trade(json),
        'bio' : filter_bio(json),
        'vegan' : filter_vegan(json), 
        'vegetarian' : filter_vegetarian(json),
        'low-food-processing' : filter_low_food_processing(json),
        'nutri-score' : filter_nutri_score(json),
        'packaging' : filter_packaging(json),
        'transport' : filter_transport(json),
     }

def create_metric(eans):
    
    return_json = json.loads(post_ean_list(eans))
    
    keys = list(return_json.keys())
        
    return {
        'palm-oil-free' : [filter_palm_oil_free(return_json[key]) for key in keys],
        'fair-trade' : [filter_fair_trade(return_json[key]) for key in keys],
        'bio' : [filter_bio(return_json[key]) for key in keys],
        'vegan' : [filter_vegan(return_json[key]) for key in keys], 
        'vegetarian' : [filter_vegetarian(return_json[key]) for key in keys],
        'low-food-processing' : [filter_low_food_processing(return_json[key]) for key in keys],
        'nutri-score' : [filter_nutri_score(return_json[key]) for key in keys],
        'packaging' :[filter_packaging(return_json[key]) for key in keys],
        'transport' : [filter_transport(return_json[key]) for key in keys],
    }
    
def metric_to_scale(metric_json):

     return {
        'palm-oil-free' : sum(metric_json['palm-oil-free']),
        #'fair-trade' : [filter_fair_trade(return_json[key]) for key in keys],
        #'bio' : [filter_bio(return_json[key]) for key in keys],
        #'vegan' : [filter_vegan(return_json[key]) for key in keys], 
        #'vegetarian' : [filter_vegetarian(return_json[key]) for key in keys],
        #'low-food-processing' : [filter_low_food_processing(return_json[key]) for key in keys],
        #'nutri-score' : [filter_nutri_score(return_json[key]) for key in keys],
        #'packaging' :[filter_packaging(return_json[key]) for key in keys],
        #'transport' : [filter_transport(return_json[key]) for key in keys],
    }

## Driver Code

In [118]:
if __name__ == '__main__':
    print('running as Script')

running as Script


Test Single Endpoint

In [119]:
    # generate Random Shopping Trip
    eans = generateRandomShoppingList(min_len=5,max_len=10)
    # print(eans[0])
    
    # Test single EAN endpoint
    # for ean in eans:
    #    print(requests.get(f'http://127.0.0.1:8000/fetch/EAN/{ean}/').status_code)
    


Test JSON Batch Enpoint

In [120]:
    # Test batch processing endpoint
    # post_ean_list(eans)

Test Filters

In [121]:
    # palmoil
    # ean = 8412170028579
    
    # fairtrade
    # ean = 8711327373105
    
    # bio
    # ean = 3760020507350
    # resp = urlopen(f'http://127.0.0.1:8000/fetch/EAN/{ean}/')
    # return_json = json.loads(resp.read())  
        
    metric_json = create_metric(eans)
    
    print(metric_json)
    

{'palm-oil-free': [None, None, None, None, None, True], 'fair-trade': [False, None, None, None, None, False], 'bio': [False, None, None, None, None, False], 'vegan': [None, None, None, None, None, False], 'vegetarian': [None, None, None, None, None, False], 'low-food-processing': [None, None, None, None, None, None], 'nutri-score': [None, None, None, None, None, 'd'], 'packaging': ['1 Glasflasche', None, None, None, None, '1 glass jar, 1 plastic lid'], 'transport': [None, None, None, None, None, None]}
